# 批量规范化
## 一、目的：
* 防止过多的中间层导致学习参数向过大或过小的方向偏移
* 防止过拟合
## 二、基本公式
$$
\mathrm{BN}(\mathbf{x}) = \boldsymbol{\gamma} \odot \frac{\mathbf{x} - \hat{\boldsymbol{\mu}}_\mathcal{B}}{\hat{\boldsymbol{\sigma}}_\mathcal{B}} + \boldsymbol{\beta}
（注意scale:\boldsymbol{\gamma}，shift:\boldsymbol{\beta}）
$$

**其中分母常加上eps（即标准差加上一个超参数）为了使其非零化**
## 三、不同层的处理
* 全连接层：
$$
\mathbf{h} = \phi(\mathrm{BN}(\mathbf{W}\mathbf{x} + \mathbf{b}) ).
$$
* 卷积层：
  对卷积层的不同通道进行BatchNorm处理，每个通道具有自己的scale和shifting，假设批量满足
  $shape \in R^{m*h*p*q}$ 则对于每一个通道来说都要对其m*p*q个元素同时BatchNorm处理
## 四、BatchNorm2d()
### 1.参数说明：
* num_features：一般输入参数为batch_size*num_features*height*width，即为其中特征的数量
* eps：分母中添加的一个值，目的是为了计算的稳定性，默认为：1e-5
* momentum：一个用于运行过程中均值和方差的一个估计参数（我的理解是一个稳定系数，类似于SGD中的momentum的系数）
* affine：当设为true时，会给定可以学习的系数矩阵gamma和beta

In [1]:
import torch
import torch.nn as nn

net=nn.BatchNorm2d(2,affine=True)
input=torch.randn(1,2,3,4)
output=net(input)
input,output

(tensor([[[[-1.9394, -0.1252,  0.3066,  0.1768],
           [ 1.2537,  1.4705,  1.6851, -0.8037],
           [ 0.4385,  0.4111, -1.0232,  0.4801]],
 
          [[-0.4614, -0.7486, -0.6783,  2.0137],
           [-0.4156, -0.7497, -0.7075, -1.7105],
           [-0.8317,  1.4097,  1.2497, -1.7021]]]]),
 tensor([[[[-2.1068, -0.3154,  0.1109, -0.0172],
           [ 1.0461,  1.2602,  1.4721, -0.9853],
           [ 0.2412,  0.2142, -1.2021,  0.2822]],
 
          [[-0.1608, -0.4122, -0.3506,  2.0057],
           [-0.1207, -0.4131, -0.3763, -1.2541],
           [-0.4849,  1.4770,  1.3370, -1.2468]]]],
        grad_fn=<NativeBatchNormBackward0>))

### 2.源码解析：

In [ ]:
def batch_norm(X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 通过is_grad_enabled来判断当前模式是训练模式还是预测模式
    if not torch.is_grad_enabled():
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 使用全连接层的情况，计算特征维上的均值和方差
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。
            # 这里我们需要保持X的形状以便后面可以做广播运算
            mean = X.mean(dim=(0, 2, 3), keepdim=True)
            var = ((X - mean) ** 2).mean(dim=(0, 2, 3), keepdim=True)
        # 训练模式下，用当前的均值和方差做标准化
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta  # 缩放和移位
    return Y, moving_mean.data, moving_var.data

class BatchNorm(nn.Module):
    # num_features：完全连接层的输出数量或卷积层的输出通道数。
    # num_dims：2表示完全连接层，4表示卷积层
    def __init__(self, num_features, num_dims):
        super().__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成1和0
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 非模型参数的变量初始化为0和1
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.ones(shape)

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var
        # 复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # 保存更新过的moving_mean和moving_var
        Y, self.moving_mean, self.moving_var = batch_norm(
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9)
        return Y